# Introduction to HuggingFace 🤗 Transformers

This notebook introduces the basics of using HuggingFace Transformers library for natural language processing tasks.

## What you'll learn:
- Installing and setting up HuggingFace Transformers
- Loading pre-trained models and tokenizers
- Basic text classification
- Text generation
- Working with pipelines

## 1. Installation

In [ ]:
# Install required packages
!pip install transformers datasets torch torchvision torchaudio accelerate
!pip install sentencepiece  # For some tokenizers

In [ ]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import pipeline
import numpy as np
import pandas as pd

## 2. Using Pipelines (Easiest Way)

In [ ]:
# Text Classification Pipeline
classifier = pipeline("text-classification")
result = classifier("I love this movie! It's amazing.")
print(f"Classification result: {result}")

In [ ]:
# Sentiment Analysis with specific model
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

texts = [
    "I'm so happy today!",
    "This is terrible.",
    "The weather is okay, I guess."
]

results = sentiment_pipeline(texts)
for text, result in zip(texts, results):
    print(f"Text: '{text}'")
    print(f"Sentiment: {result['label']} (confidence: {result['score']:.3f})")
    print("-" * 50)

In [ ]:
# Text Generation Pipeline
generator = pipeline("text-generation", model="gpt2")
prompt = "The future of artificial intelligence is"
generated = generator(
    prompt,
    max_length=100,
    num_return_sequences=2,
    temperature=0.7,
    do_sample=True
)

for i, gen in enumerate(generated):
    print(f"Generation {i+1}:")
    print(gen['generated_text'])
    print("\n" + "="*50 + "\n")

## 3. Manual Model and Tokenizer Loading

In [ ]:
# Load a pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

print(f"Model: {model_name}")
print(f"Tokenizer vocab size: {len(tokenizer)}")
print(f"Model parameters: {model.num_parameters():,}")

In [ ]:
# Tokenize text
text = "Hello, how are you today?"
tokens = tokenizer(text, return_tensors="pt")

print(f"Original text: {text}")
print(f"Input IDs shape: {tokens['input_ids'].shape}")
print(f"Input IDs: {tokens['input_ids']}")
print(f"Tokens: {tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])}")

In [ ]:
# Get model embeddings
with torch.no_grad():
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state

print(f"Embeddings shape: {embeddings.shape}")
print(f"First token embedding (first 10 dimensions): {embeddings[0, 0, :10]}")

## 4. Question Answering Example

In [ ]:
# Question Answering Pipeline
qa_pipeline = pipeline("question-answering")

context = """
The Transformer architecture was introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017.
It revolutionized natural language processing by using self-attention mechanisms instead of recurrent or 
convolutional layers. The model consists of an encoder and decoder, each made up of multiple layers.
BERT, GPT, and T5 are all based on the Transformer architecture.
"""

questions = [
    "When was the Transformer architecture introduced?",
    "What paper introduced the Transformer?",
    "What models are based on the Transformer architecture?"
]

for question in questions:
    result = qa_pipeline(question=question, context=context)
    print(f"Question: {question}")
    print(f"Answer: {result['answer']} (confidence: {result['score']:.3f})")
    print("-" * 60)

## 5. Named Entity Recognition (NER)

In [ ]:
# Named Entity Recognition
ner_pipeline = pipeline("ner", aggregation_strategy="simple")

text = "Apple Inc. was founded by Steve Jobs in Cupertino, California in 1976."
entities = ner_pipeline(text)

print(f"Text: {text}\n")
print("Named Entities:")
for entity in entities:
    print(f"- {entity['word']}: {entity['entity_group']} (confidence: {entity['score']:.3f})")

## 6. Working with Different Model Types

In [ ]:
# Compare different models for sentiment analysis
models_to_compare = [
    "distilbert-base-uncased-finetuned-sst-2-english",
    "cardiffnlp/twitter-roberta-base-sentiment-latest"
]

test_text = "I'm not sure if I like this new update."

for model_name in models_to_compare:
    try:
        classifier = pipeline("sentiment-analysis", model=model_name)
        result = classifier(test_text)
        print(f"Model: {model_name}")
        print(f"Result: {result[0]}")
        print("-" * 60)
    except Exception as e:
        print(f"Error with {model_name}: {e}")

## 7. Batch Processing

In [ ]:
# Batch processing for efficiency
texts = [
    "This movie is fantastic!",
    "I didn't enjoy this film at all.",
    "The plot was confusing but the acting was good.",
    "Best movie I've seen this year!",
    "Boring and predictable."
]

# Process all texts at once
classifier = pipeline("sentiment-analysis")
results = classifier(texts)

# Create a DataFrame for better visualization
df = pd.DataFrame({
    'text': texts,
    'sentiment': [r['label'] for r in results],
    'confidence': [r['score'] for r in results]
})

print("Batch Processing Results:")
print(df.to_string(index=False))

## 8. Device Management (GPU/CPU)

In [ ]:
# Check available devices
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.get_device_name()}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Create pipeline with specific device
device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU
classifier = pipeline("sentiment-analysis", device=device)

print(f"\nPipeline running on: {'GPU' if device >= 0 else 'CPU'}")

## Next Steps

In the following notebooks, we'll explore:
- Loading and preprocessing datasets
- Text classification fine-tuning
- Token classification (NER) fine-tuning
- Question answering fine-tuning
- Text generation fine-tuning
- Advanced techniques like LoRA
- Model evaluation and deployment

This notebook provided a foundation for working with HuggingFace Transformers. Practice with different models and tasks to get comfortable with the library!